# Модель для выбора подходящего тарифа
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Нужно построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
Имеются данные о поведении клиентов, которые уже перешли на эти тарифы, на основании этих данных планируется построить модель для задачи классификации, которая выберет подходящий тариф.

## 1. Изучение данных из файла 
<a id="start"></a>

In [1]:
#импорт библиотек
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier

In [2]:
#открытие файлов
try:
    df = pd.read_csv('users_behavior.csv') # чтение файла локально
except:
    df = pd.read_csv('/datasets/users_behavior.csv')  # чтение файла с сервера
    
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Данные уже подготовлены к анализу, количество звонков, их продолжительность, количество сообщений и объём трафика посчитаны. Тариф обозначен булевым значением: 0 это смарт, 1 это ультра

## 2.  Разделение данных
<a id="split"></a>

Теперь нужно разделить данные на целевой признак и остальные данные, а эти группы в свою очередь на тренировочные, валидационные и тестовые. В курсе рекомендовано соотношение: 60% тренировочные данные, 20% валидационные, 20% тестовые.

In [3]:
features = df.drop(['is_ultra'], axis=1)# извлечение признаков
target = df['is_ultra'] # извлечение  целевого признака

features_train, features_test_and_valid, target_train, target_test_and_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target) # разделение данных на тренировочные и валидационные+тестовые
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test_and_valid, target_test_and_valid, test_size=0.5, random_state=12345, stratify=target_test_and_valid) 
# разделение данных на валидационные и тестовые


## 3. Исследование качества моделей
<a id="models"></a>

Нам предстоит исследовать качество разных моделей в отношении задачи классификации, а затем проверить его на тестовой выборке.

#### Дерево решений
<a id="tree"></a>

In [4]:
#decisiontree
for i in range(1, 6):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=i) # инициализация модели с заданной глубиной дерева
    model_1.fit(features_train, target_train) # обучение модели
    predictions_valid = model_1.predict(features_valid)# предсказания модели
    print("max_depth =", i, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))


max_depth = 1 : 0.7402799377916018
max_depth = 2 : 0.7729393468118196
max_depth = 3 : 0.7776049766718507
max_depth = 4 : 0.7542768273716952
max_depth = 5 : 0.7853810264385692


In [5]:
#Сравним метрики для обучающей и валидационной выборок
result1 = model_1.score(features_train, target_train)
result2 = model_1.score(features_valid, target_valid)
print("Accuracy модели на обучающей выборке:", result1)
print("Accuracy модели на валидационной выборке:", result2)

Accuracy модели на обучающей выборке: 0.8137966804979253
Accuracy модели на валидационной выборке: 0.7853810264385692


Лучший показатель оказался у модели с максимальной глубиной дерева 3. Но нужно посмотреть на другие алгоритмы, кроме того, тестовая выборка рассудит.
Как и свойственно деревьям решений, эта модель склонна к переобучению, точность предсказаний на тренировочной выборке выше, чем на валиационной, но не намного.

#### Случайный лес
<a id="forest"></a>

In [18]:
#random forest
model_2 = None
best_result = 0
for est in range(40, 60):
    for d in range(1, 6):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = d) # обучение модели с заданным количеством деревьев
        model.fit(features_train, target_train) # обучение модели на тренировочной выборке
        result = model.score(features_valid, target_valid) # оценка качества модели на валидационной выборке
        if result > best_result:
            model_2 = model
            best_result = result
            n_est = est
            depth = d

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев", n_est, "Глубина дерева", d)

Accuracy наилучшей модели на валидационной выборке: 0.8258164852255054 Количество деревьев 43 Глубина дерева 5


In [19]:
#Сравним метрики для обучающей и валидационной выборок
result1 = model_2.score(features_train, target_train)
result2 = model_2.score(features_valid, target_valid)
print("Accuracy модели на обучающей выборке:", result1)
print("Accuracy модели на валидационной выборке:", result2)

Accuracy модели на обучающей выборке: 0.8164138467522365
Accuracy модели на валидационной выборке: 0.8258164852255054


43 дерева пока выдали лучший результат 0.825. Если перебирать количество деревьев от 10 до 200 с разными шагами, а максимальную глубину от 1 до 5, оптимальный результат получается на уровне 43 деревьев и максимальной глубины 5. И как ни станно, модель даже не переобучена.

#### Логистическая регрессия
<a id="regr"></a>

In [8]:
#logisticregression

model_3 = LogisticRegression(random_state=12345)# инициализация модели логистической регрессии с параметром random_state=12345
model_3.fit(features_train, target_train) # обучение модели на тренировочной выборке

result = model_3.score(features_valid, target_valid) # получение метрики качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.71850699844479


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
#Сравним метрики для обучающей и валидационной выборок
result1 = model_3.score(features_train, target_train)
result2 = model_3.score(features_valid, target_valid)
print("Accuracy модели на обучающей выборке:", result1)
print("Accuracy модели на валидационной выборке:", result2)

Accuracy модели на обучающей выборке: 0.7105809128630706
Accuracy модели на валидационной выборке: 0.71850699844479


Логистическая регрессия явно проигрывает остальным моделям, но зато не склонна к переобучению

## 4. Проверка качества моделей на тестовой выборке
<a id="test"></a>

In [20]:
# новая обучающая выборка
print(features_train.shape, features_valid.shape, target_train.shape, target_valid.shape)
features_train = pd.concat([features_train, features_valid], ignore_index=True)
target_train = pd.concat([target_train, target_valid], ignore_index=True)
#display(features_train)
#display(target_train)
#print(features_train.shape, features_valid.shape, target_train.shape, target_valid.shape)
model_2.fit(features_train, target_train) #обучение модели с помощью новой обучающей выборки

(2571, 4) (643, 4) (2571,) (643,)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=43,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [21]:
predictions_test = model_2.predict(features_test) # предсказания модели на тестовой выборке
print(accuracy_score(target_test, predictions_test))

0.8273716951788491


Качество модели на тестовой выборке оказалось действительно высоким. Можно ещё сравнить с второй по качеству моделью.

In [22]:
 predictions_test2 = model_1.predict(features_test)
 print(accuracy_score(target_test, predictions_test2))   

0.8118195956454122


Тоже подходит (в задании указан порог в 0,75), но уступает случайному лесу с параметром estimators = 48. 

Вывод: Лучшая по качеству модель получается из случайного леса на 43 дерева, с глубиной 5, значение метрики accuracy на тестовой выборке для неё равно 82.7, что является достаточно высоким показателем, кроме того модель не подает признаков значимого переобучения. 

## 5. Дополнительное задание: проверка модели на вменяемость
<a id="sanity"></a>

In [23]:
#создадим молель-пустышку, которая выдает самый распространенный в выборке класс
model_d = DummyClassifier(strategy='most_frequent', random_state=0) 
model_d.fit(features_train, target_train) # обучим её с помощью обучающей выборки
predictions_valid = model_d.predict(features_valid) # запишем предсказания
print(accuracy_score(target_valid, predictions_valid)) # проверим их точность

0.6936236391912908


Ну, кажется наша модель оказалась качественнее пустышки-классификатора, которая выдает только нули в качестве ответов

## Вывод
Построена модель, которая на основании данных о совершенных звонках, использовании интернета и отправленных смс с точностью 79% предсказывает какой тариф больше подходит пользователю смарт или ультра. Модель проверена на вменяемость, её точность заметно больше, чем точность модели-пустышки. Параметры лучшей модели - случайный лес с количеством деревьев, равным 50, он дает наибольшую точность и его работа не занимает много времени.